<a href="https://colab.research.google.com/github/Yasserashraf1/AI-Salaries-Analysis-Project-Using-R-and-Tableau/blob/main/NextGen_HR_FCDS_prototype_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import pandas as pd
import os
import pickle
import random
from sklearn.metrics import accuracy_score

# Initialize the database with the new structure and add 300 dummy records
def initialize_database():
    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS employees')  # Drop the table if it exists
    cursor.execute('''CREATE TABLE employees (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        age INTEGER,
                        experience INTEGER,
                        skills TEXT,
                        department TEXT,
                        salary INTEGER,
                        performance_status TEXT)''')

    # Generate 300 dummy records
    departments = ['HR', 'Engineering', 'Marketing', 'Sales', 'Finance']
    skills = ['Python', 'Java', 'Excel', 'SQL', 'Leadership', 'Communication']

    for _ in range(300):
        name = f"Employee_{random.randint(1000, 9999)}"
        age = random.randint(22, 50)
        experience = random.randint(1, 25)
        skill_set = random.sample(skills, random.randint(1, len(skills)))  # Random skills
        skills_text = ', '.join(skill_set)
        department = random.choice(departments)
        salary = random.randint(3000, 15000)
        performance_status = random.choice(['Accept', 'Reject'])

        cursor.execute('''INSERT INTO employees (name, age, experience, skills, department, salary, performance_status)
                          VALUES (?, ?, ?, ?, ?, ?, ?)''',
                       (name, age, experience, skills_text, department, salary, performance_status))

    conn.commit()
    conn.close()
    print("Database initialized and 300 dummy records added!")

# Manual HR functions (as before)
def add_employee():
    name = input("Enter name: ")
    age = int(input("Enter age: "))
    experience = int(input("Enter years of experience: "))
    skills = input("Enter skills (comma-separated): ")
    department = input("Enter department: ")
    salary = int(input("Enter salary: "))
    performance_status = input("Enter performance status (Accept/Reject): ")

    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO employees (name, age, experience, skills, department, salary, performance_status)
                      VALUES (?, ?, ?, ?, ?, ?, ?)''',
                   (name, age, experience, skills, department, salary, performance_status))
    conn.commit()
    conn.close()
    print("Employee added successfully!")

def delete_employee():
    emp_id = int(input("Enter employee ID to delete: "))
    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM employees WHERE id = ?', (emp_id,))
    conn.commit()
    conn.close()
    print("Employee deleted successfully!")

def view_employees():
    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM employees')
    rows = cursor.fetchall()
    conn.close()
    for row in rows:
        print(row)

def update_employee():
    emp_id = int(input("Enter employee ID to update: "))
    name = input("Enter new name: ")
    age = int(input("Enter new age: "))
    experience = int(input("Enter new years of experience: "))
    skills = input("Enter new skills (comma-separated): ")
    department = input("Enter new department: ")
    salary = int(input("Enter new salary: "))
    performance_status = input("Enter new performance status (Accept/Reject): ")

    conn = sqlite3.connect('employee.db')
    cursor = conn.cursor()
    cursor.execute('''UPDATE employees
                      SET name = ?, age = ?, experience = ?, skills = ?, department = ?, salary = ?, performance_status = ?
                      WHERE id = ?''',
                   (name, age, experience, skills, department, salary, performance_status, emp_id))
    conn.commit()
    conn.close()
    print("Employee updated successfully!")

# Train the ML model
def train_ml_model():
    conn = sqlite3.connect('employee.db')
    df = pd.read_sql_query("SELECT * FROM employees", conn)
    conn.close()

    if len(df) < 10:  # Minimum data requirement
        print("Not enough data to train the model. Add more employees.")
        return

    # Feature columns
    X = df[['age', 'experience', 'salary']].values
    y = (df['performance_status'] == 'Accept').astype(int).values

    # Split data for training the acceptance prediction model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Random Forest model for acceptance/rejection
    acceptance_model = RandomForestClassifier(random_state=42)
    acceptance_model.fit(X_train, y_train)

    # Save the model
    with open('acceptance_model.pkl', 'wb') as f:
        pickle.dump(acceptance_model, f)

    y_pred = acceptance_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model trained and saved successfully with accuracy: {accuracy:.2f}")

# AI-Powered HR Mode
def ai_hr_mode():
    if not os.path.exists('acceptance_model.pkl'):
        print("No trained model found. Train the model first.")
        return

    # Load the trained model
    with open('acceptance_model.pkl', 'rb') as f:
        acceptance_model = pickle.load(f)

    print("Enter new employee details:")
    age = int(input("Enter age: "))
    experience = int(input("Enter years of experience: "))
    desired_salary = int(input("Enter desired salary: "))
    skills = input("Enter skills (comma-separated): ")
    required_skills = ['Python', 'SQL']  # Example: Required skills for the company

    # Check if skills match
    skill_list = skills.split(", ")
    skill_match = any(skill in required_skills for skill in skill_list)

    # Predict acceptance
    prediction = acceptance_model.predict([[age, experience, desired_salary]])

    if prediction[0] == 1 and skill_match:
        print("AI Recommendation: Accept")
        print(f"Reason for Acceptance: The candidate matches the age, experience, salary expectations, and required skills.")
    else:
        print("AI Recommendation: Reject")
        reason = "Skills mismatch" if not skill_match else "Insufficient qualifications"
        print(f"Reason for Rejection: {reason}")

# Main function
def main():
    initialize_database()  # Ensure database is set up and add dummy data
    while True:
        print("\n1. Manual HR Mode")
        print("2. AI-Powered HR Mode")
        print("3. Train ML Model")
        print("4. Exit")
        choice = int(input("Enter your choice: "))

        if choice == 1:
            print("\nManual HR Mode")
            print("1. Add Employee")
            print("2. Delete Employee")
            print("3. View Employees")
            print("4. Update Employee")
            manual_choice = int(input("Enter your choice: "))

            if manual_choice == 1:
                add_employee()
            elif manual_choice == 2:
                delete_employee()
            elif manual_choice == 3:
                view_employees()
            elif manual_choice == 4:
                update_employee()
            else:
                print("Invalid choice!")
        elif choice == 2:
            ai_hr_mode()
        elif choice == 3:
            train_ml_model()
        elif choice == 4:
            print("Exiting the program.")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


Database initialized and 300 dummy records added!

1. Manual HR Mode
2. AI-Powered HR Mode
3. Train ML Model
4. Exit
Enter your choice: 4
Exiting the program.


In [ ]:
pip install tabulate


In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
pip install pyngrok

In [ ]:
import sqlite3
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate  # Import tabulate library
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize database and load CSV
def initialize_database():
    con = sqlite3.connect('HR.db')
    cur = con.cursor()

    # Create table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS EmployeesRecords (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Age INTEGER,
            Accessibility INTEGER,
            EducationLevel TEXT,
            Gender TEXT,
            WorkedBefore INTEGER,
            MentalHealth TEXT,
            MainBranch TEXT,
            YearsOfCoding INTEGER,
            YearsOfCodingWhileWorking INTEGER,
            Country TEXT,
            PreviousSalary REAL,
            HaveWorkedWith TEXT,
            ComputerSkills INTEGER,
            Employed INTEGER
        );
    """)

    # Import CSV
    with open("stackoverflow_full .csv", 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Skip the header row
        rows_to_insert = []
        for count, row in enumerate(reader):
            if count >= 50000:  # Limit to 50,000 rows
                break
            rows_to_insert.append(row[1:])  # Skip the first column (assuming it's an ID column)
        cur.executemany(f"""
            INSERT INTO EmployeesRecords (
                Age, Accessibility, EducationLevel, Gender, WorkedBefore, MentalHealth,
                MainBranch, YearsOfCoding, YearsOfCodingWhileWorking, Country, PreviousSalary,
                HaveWorkedWith, ComputerSkills, Employed
            )
            VALUES ({','.join(['?'] * (len(headers) - 1))});
        """, rows_to_insert)

    con.commit()
    con.close()
    print("Database initialized and CSV data imported successfully!")

# Retrieve Data from DB to DataFrame for EDA
def load_data_to_df():
    con = sqlite3.connect('HR.db')
    df = pd.read_sql_query("SELECT * FROM EmployeesRecords", con)
    con.close()
    return df

# Pretty-print DataFrame as a table
def print_pretty_table(df):
    print("\n" + tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))

# EDA & Data Preprocessing
def perform_eda_and_preprocessing(df):
    print("\n--- EDA & Data Preprocessing ---")

    # Checking for missing values
    print("\nMissing Values:")
    print(df.isnull().sum())

    # Dropping duplicate rows
    print("\nDropping duplicates...")
    df = df.drop_duplicates()

    # Encoding categorical variables
    label_encoder = LabelEncoder()
    categorical_columns = ['Age', 'Accessibility', 'EducationLevel', 'WorkedBefore', 'Gender', 'MentalHealth', 'MainBranch', 'Country']

    for col in categorical_columns:
        df[col] = label_encoder.fit_transform(df[col].astype(str))

    # Handle outliers (IQR method)
    def remove_outliers_iqr(data, column_name):
        Q1 = data[column_name].quantile(0.25)
        Q3 = data[column_name].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        data = data[(data[column_name] >= lower_bound) & (data[column_name] <= upper_bound)]
        return data

    # Remove outliers
    for col in ['YearsOfCoding', 'PreviousSalary', 'ComputerSkills']:
        df = remove_outliers_iqr(df, col)

    # Feature and target selection
    X = df[['Age', 'Accessibility', 'ComputerSkills', 'PreviousSalary']]
    y = df['Employed']

    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("\nEDA and Preprocessing complete.")
    return X_train, X_test, y_train, y_test

# Train the models and print results
def train_models(X_train, X_test, y_train, y_test):
    print("\n--- Training Models ---")

    # Logistic Regression
    logistic_regression = LogisticRegression(max_iter=1000)
    logistic_regression.fit(X_train, y_train)
    y_pred_lr = logistic_regression.predict(X_test)
    accuracy_lr = accuracy_score(y_test, y_pred_lr)
    print(f"Logistic Regression Accuracy: {accuracy_lr:.2f}")

    # Decision Tree
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(X_train, y_train)
    y_pred_dt = decision_tree.predict(X_test)
    accuracy_dt = accuracy_score(y_test, y_pred_dt)
    print(f"Decision Tree Accuracy: {accuracy_dt:.2f}")

    # Random Forest
    random_forest = RandomForestClassifier()
    random_forest.fit(X_train, y_train)
    y_pred_rf = random_forest.predict(X_test)
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    print(f"Random Forest Accuracy: {accuracy_rf:.2f}")

    # Store models in a dictionary and return
    models = {
        'Logistic Regression': logistic_regression,
        'Decision Tree': decision_tree,
        'Random Forest': random_forest
    }
    return models

# AI-Powered HR Mode: Model selection and prediction
def ai_hr_mode(models):
    print("\n--- AI-Powered HR Mode ---")
    print("Available Models:")
    for i, model_name in enumerate(models.keys(), start=1):
        print(f"{i}. {model_name}")
    choice = int(input("Select a model by number: "))
    model_name = list(models.keys())[choice - 1]
    model = models[model_name]
    print(f"Using {model_name} for predictions")

    # Get input for prediction
    age = int(input("Age: "))
    accessibility = int(input("Accessibility (1 or 0): "))
    computer_skills = int(input("Computer Skills "))
    previous_salary = float(input("Previous Salary: "))

    # Create a DataFrame for prediction with the same column names as the training data
    input_data = pd.DataFrame([[age, accessibility, computer_skills, previous_salary]],
                              columns=['Age', 'Accessibility', 'ComputerSkills', 'PreviousSalary'])

    # Predict
    prediction = model.predict(input_data)[0]
    print(f"Prediction Result: {'Accepted' if prediction == 1 else 'Rejected'}")

# Manual HR Mode - Retrieve number of head rows
def retrieve_head_rows():
    con = sqlite3.connect('HR.db')
    df = pd.read_sql_query("SELECT * FROM EmployeesRecords", con)
    con.close()

    num_rows = int(input("Enter the number of rows you want to retrieve: "))
    print(f"\nFirst {num_rows} rows from the database:")
    print_pretty_table(df.head(num_rows))  # Pretty print the head rows

# Main function to interact with user
def main():
    models = {}  # To store trained models and avoid retraining
    while True:
        print("\n1. Manual HR Mode")
        print("2. AI-Powered HR Mode")
        print("3. Train Models")
        print("4. Exit")
        choice = int(input("Enter your choice: "))

        if choice == 1:
            print("\nManual HR Mode")
            print("1. Add Employee")
            print("2. Retrieve Employee")
            print("3. Delete Employee")
            print("4. Update Employee")
            print("5. Retrieve Head Rows from DB")  # New option
            manual_choice = int(input("Enter your choice: "))
            if manual_choice == 1:
                add_employee()
            elif manual_choice == 2:
                retrieve_employee()
            elif manual_choice == 3:
                delete_employee()
            elif manual_choice == 4:
                update_employee()
            elif manual_choice == 5:
                retrieve_head_rows()  # New option action
            else:
                print("Invalid choice!")

        elif choice == 2:
            if not models:  # If models haven't been trained yet
                print("Please train the models first (Option 3)!")
            else:
                # AI Mode - Using pre-trained models
                ai_hr_mode(models)

        elif choice == 3:
            # Load data to DataFrame and train models only once
            df = load_data_to_df()
            X_train, X_test, y_train, y_test = perform_eda_and_preprocessing(df)
            models = train_models(X_train, X_test, y_train, y_test)

        elif choice == 4:
            print("Exiting the program.")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()



1. Manual HR Mode
2. AI-Powered HR Mode
3. Train Models
4. Exit
Enter your choice: 3

--- EDA & Data Preprocessing ---

Missing Values:
ID                           0
Age                          0
Accessibility                0
EducationLevel               0
Gender                       0
WorkedBefore                 0
MentalHealth                 0
MainBranch                   0
YearsOfCoding                0
YearsOfCodingWhileWorking    0
Country                      0
PreviousSalary               0
HaveWorkedWith               0
ComputerSkills               0
Employed                     0
dtype: int64

Dropping duplicates...

EDA and Preprocessing complete.

--- Training Models ---
Logistic Regression Accuracy: 0.77
Decision Tree Accuracy: 0.90
Random Forest Accuracy: 0.90

1. Manual HR Mode
2. AI-Powered HR Mode
3. Train Models
4. Exit
Enter your choice: 2

--- AI-Powered HR Mode ---
Available Models:
1. Logistic Regression
2. Decision Tree
3. Random Forest
Select a model by num